# NLP Topic Modeling in Python with Non-negative Matrix Factorization 

## Using Jupyter Notebooks 

Jupyter Notebooks is an interactive Python environment for data science.   Cells are seperated into Markdown (i.e., text) and code cells.  In this notebook, you should not need to edit code (unless you really want to!). Therefore, you can just run each cell by highlighting it and pressing "Cmd + Return" or using the "> Run" key at the top.

Note: A best practice is to import packages in the first cell of the notebook.  However, given that this is a tutorial I will import them in the first cell in which they are used to more closely associate the package with it's use.  

## Installing Packages 
First, we'll install some packages we'll use today for our text manipulation and topic modeling. This may produce a lot of output so please be patient. 

In [ ]:
!pip install nltk

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

## 1. Import our Data

We're going to use pandas to import and inspect our data.  Notice that our text column is already in lower case and contains the article text from wikipedia.  

In [1]:
# Import Data 
import pandas as pd 
df = pd.read_csv('people_wiki.csv')

#Inspect our dataframe
df.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## 2. Pre-process our Text 

Before we analyze our text we need to clean it.  Cleaning text involves standardizing and removing terms that are non-informative.  Terms that occur in most documents or, alternatively, very few, are unlikely to help us know how to group things. 

In [37]:
import re
import string
from nltk.tokenize.regexp import WordPunctTokenizer
from nltk.corpus import stopwords


# These lines determines what the punctuation and numbers are replaced with.
punct_table = str.maketrans({ch: ' ' for ch in string.punctuation})  
digit_table = str.maketrans('', '', string.digits)

# Text cleaning functions
remove_punctuation = lambda x: x.translate(punct_table)
remove_numbers = lambda x: x.translate(digit_table)
remove_urls = lambda x: re.sub(r"http\S+", "", x)


#Tokenize texts.  Note- It is possible to comment out steps with a # to change how tokenization occurs. 
def tokenize(text):
    """
    Takes a list of strings and return 
         
    """
    # Creates stopword list from NLTK.
    sw = stopwords.words("english") + ['']
    
    # Creates a tokenizer instance. 
    tokenizer = WordPunctTokenizer() 
    
    # Text cleaning. 
    text = remove_urls(text) # removes urls 
    text = remove_punctuation(text)
    text = remove_numbers(text) # removes numbers.  Leaving here as dates may be informative. 
    text = text.lower() # sets to lowercase

    # Tokenization 
    tokenizer = WordPunctTokenizer()
    tkns = tokenizer.tokenize(text) # tokenizes text

    # Remove stopwords 
    tokenized_text = [tkn for tkn in tkns if tkn not in sw]

    return tokenized_text 

Let's see a demonstration of what the above function is doing! 

In [38]:
text = "I moved to Kansas City in 2020 during the pandemic!  It was hard to find housing " \
       "with social distancing but I was eventually able to find one on Zillow (http://www.zillow.com)."

tokenize(text)

['moved',
 'kansas',
 'city',
 'pandemic',
 'hard',
 'find',
 'housing',
 'social',
 'distancing',
 'eventually',
 'able',
 'find',
 'one',
 'zillow']

### Feature Extraction (Vectorization) 

Computers only undestand numbers we need to convert the tokenized documents into vectors.  To do this we'll use term frequency - inverse docuemnt frequency (TF-IDF) metric. 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=tokenize, max_df=.95, min_df=.0001, ngram_range=(1,2))
doc_term_matrix = vectorizer.fit_transform(list(df['text']))

print("Our matrix has {} documents and {} vocbaulary terms.".format(doc_term_matrix.shape[0], 
                                                                    doc_term_matrix.shape[1]))

# Store our vocab for later use. 
model_features = vectorizer.get_feature_names_out()

print("Note that our model features list has the same length as the vocabulary (i.e, they are the same.): {} \n".format(len(model_features)))
print("Sample Feature Names: ", model_features[100:110])

Our matrix has 42786 documents and 212455 vocbaulary terms.
Note that our model features list has the same length as the vocabulary (i.e, they are the same.): 212455 

Sample Feature Names:  ['abbotsford' 'abbott' 'abbott college' 'abbotts' 'abboud' 'abbreviated'
 'abby' 'abc' 'abc affiliate' 'abc also']


# 3.  Model Creation:  Non-Negative Matrix Factorization (NMF)

Shout out to Hui for teaching me that this could be used for topic modeling!  We are going to keep things very simple but I want to provide just a superficial explanation of the logic of this technique.  

Non-negative matrix factorization (NMF) seems like an intimidating technique but the basic logic is very simple. You may remember from elementary school that factorization is breaking a number down into numbers that when multiplied together equal the initial value (e.g., 30 = (2X3)X5 ).  We can do something similiar with matrices.  However, matrix multiplication has a special requirement that the number of rows in one matrix (we'll call is matrix W) must equal the number of columns in the other matrix (matrix H).  Because there are many different values that the number of rows/number of columns can take on we have to select a value for factorization.  Matrix multiplication follows slighty different rules from traditional multiplication which you can read about here.    

Just like we can multiply a simple factorization back together to get the original value (e.g., (2x3)x5 = 30), we can multiply Matrix W X Matrix H to reconstruct our original Document X Term matrix.  However, our reconstructed matrix is unlikely to exactly match our original matrix and the degree to which the values differ tells us how well our proposed model fits.  

One last thing!  *Non-negative Matrix Factorization is exactly that - non-negative*.  No value in the matrix can be below zero.  This makes it a good fit for behavioral data (you can't have negative clicks) but a poor fit for things like financial data where there may be negative values.  

In [9]:
# Enter the number of topics to model. 
num_topics = 100

from sklearn.decomposition import NMF

# Initialize model
model = NMF(init='nndsvd', n_components=num_topics)

# Fit our corpus to the model 
model.fit(doc_term_matrix)

# Get document weights for each component. 
doc_weights = model.transform(doc_term_matrix)

# 4.  Model Inspection 

### Retrieve Top Terms for each Topic

Here we are retrieving the topic terms that most characterize each topic.  Note that all terms are scored for each topic but we are interested in terms that are most unique and typical for a given a topic.   Generally we inspect the top N terms to get an idea about what the topic is.  I've provided an easy way for you to vary the number of terms returned below. 

In [44]:
#How many terms to do you want to retrieve? 
n_terms = 10

def get_nmf_topics(model, n_top_words, num_topics, feat_names):
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i)] = words;
    
    return pd.DataFrame(word_dict)

nmf_topics = get_nmf_topics(model, n_terms, model.n_components, model_features)
nmf_topics

,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,university,league,records,liberal,film,league,championships,art,orchestra,united states,...,member,jewish,physics,state,foreign,school,european,french,texas,computer
1,professor,season,record,labor,films,baseball,metres,museum,symphony,states,...,committee,israel,theory,state university,affairs,high,german,france,austin,engineering
2,studies,club,producer,election,directed,major league,medal,gallery,conductor,united,...,board,rabbi,mathematical,ohio,united nations,high school,european parliament,paris,university texas,science
3,college,town,dj,seat,feature,major,world,artist,symphony orchestra,district,...,chairman,israeli,mathematics,florida,nations,attended,parliament,le,houston,software
4,phd,side,label,assembly,documentary,pitcher,athletics,arts,philharmonic,us,...,council,hebrew,quantum,ohio state,ambassador,education,polish,des,dallas,technology
5,faculty,goals,hip,legislative,director,league baseball,world championships,work,conducting,states district,...,served,jerusalem,theoretical,university,international,college,germany,et,texas austin,computer science
6,education,appearances,hop,liberal party,film festival,era,silver,exhibition,chamber,district court,...,advisory,palestinian,professor,michigan,security,graduated,european tour,du,san antonio,systems
7,degree,football,hip hop,legislative assembly,cinema,pitched,bronze,contemporary,piano,attorney,...,chair,jews,sciences,nigeria,secretary,teacher,member,de,antonio,information
8,visiting,manager,artists,party,feature film,innings,gold,painting,orchestras,states senate,...,education,tel,physicist,governor,united,year,member european,les,american,computing
9,academic,signed,produced,labor party,short,season,olympic,paintings,music director,private practice,...,association,aviv,prize,secretary state,political,schools,dutch,quebec,houston texas,electrical


### Inspecting Representative Documents

We typically want to be able to inspect the original data.  Above when we fit the model, we saved weights that each component characterizes a document.  We're going to select the component with the max value for each document and assign it to that topic.  Then we'll filter our dataframe by topic to see if our topic modeling worked! 

In [11]:
print("Note that the document weights matrix has the name number of rows as documents: {}".format(doc_weights.shape[0]))
print("Additionally observe that is the same number of columns as our topics: {}".format(doc_weights.shape[1]))

Note that the document weights matrix has the name number of rows as documents: 42786
Additionally observe that is the same number of columns as our topics: 100


Now let's print some bios to inspect! Do they make sense to you?  What themes do they have? 

In [41]:
# Which topics documents do you want to inspect? 
topic_id =47

# Reformating topic number 
topic_col = 'Topic # ' + '{:02d}'.format(topic_id)

# Get topic terms 
print("These words characterize this topic: ", "\n")
print(nmf_topics[topic_col], "\n\n")

# Assign topics to biographies in the Dataframe
df["Topic_idx"] = doc_weights.argmax(axis=1)

# Filtering our dataframe. 
df_topic = df.loc[df['Topic_idx'] == topic_id] 
bios = zip(df_topic['name'], df_topic['text'])

# Displaying the selected bios. 
print("Here are the biographies for individuals who scored highly on this topic: ", '\n')

for bio in bios:
    print("Name: ", bio[0])
    print("Biography: ", bio[1], "\n\n")
    

These words characterize this topic:  

0           poker
1            wsop
2           event
3    series poker
4    world series
5           limit
6      tournament
7    poker player
8        winnings
9          holdem
Name: Topic # 47, dtype: object 


Here are the biographies for individuals who scored highly on this topic:  

Name:  Roland De Wolfe
Biography:  roland de wolfe born 1979 in london is an english professional poker player and a former writer for the poker magazine inside edgeafter winning first prize in an event at the 2004 gutshot poker festival he went on to make a money finish in the 1000 no limit event of the 2005 world series of pokerin july 2005 he won first place in the major grand prix de paris event of the world poker tour wpts fourth season defeating former champion juha helppi in the final headsup battlein april 2006 he finished 3rd out of 605 runners in the wpt 25000 championship event at the bellagio winning over 1000000 in october 2006 de wolfe won the eu

# 5. Conclusion

Hopefully you found the tutorial above interesting!  If you want to learn more about cleaning and preprocessing text as well as a different technique for topic modeling check out the Evolytics blog series here:  

- [Part II. Preparing Text for Analysis with Natural Language Toolkit (NLTK)](https://evolytics.com/blog/open-ended-survey-questions-for-computational-analysis-part-ii/)
- [Part III. How to Find Near Duplicate Text and Recognize Name Entities in Survey Responses](https://evolytics.com/blog/survey-responses-duplicate-text-and-named-entities/)

Please feel free to use the above code for your own projects.  
